In [1]:
import pandas as pd
import numpy as np
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os
import json
from pathlib import Path

# define groq key
groq_key = (
    "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"  # os.environ['groqkey']
)

In [2]:
from extraction_agents import LLMAgent
from data_query import DataLoader

/Users/yarg/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/DataSci210_MedicationDeprescriber/src/extraction_agents.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  base_embeddings = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
/opt/anaconda3/envs/capstone/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
base_path = Path(
    "/Users/yarg/Library/CloudStorage/OneDrive-Personal/Documents/GitHub/DataSci210_MedicationDeprescriber/Data"
)

In [4]:
agent = LLMAgent(groq_key=groq_key, data_path=base_path)

In [5]:
encounter_key="D8FCAB3DB25660"

In [6]:
stuff = agent.extract_diagnosis(encounter_key=encounter_key)

In [7]:
stuff

{'Mild to moderate esophagitis': False,
 'GERD': False,
 'Peptic Ulcer Disease': False,
 'Upper GI symptoms': False,
 'ICU Stress Ulcer Prophylaxis': False,
 "Barrett's Esophagus": False,
 'Chronic NSAID use with bleeding risk': False,
 'Severe esophagitis': False,
 'Documented history of bleeding GI ulcer': False,
 'Reasoning': "The provided JSON information only contains two identical entries with an EncounterKey and a DxName of '*Unspecified'. This does not provide any specific information about the patient's medical conditions, so we cannot determine if the patient has any of the listed conditions."}

In [8]:
DL = DataLoader(data_path=base_path)

In [9]:
DL.get_label(encounter_key=encounter_key)

,key,reason,recommendation
0,D6253A5CE371EA,example with gastrointestinal bleed should con...,continue
1,DFAFED1811B871,example with a documented history of a GI blee...,continue
2,D4865B8BBB294E,example with GI bleeding that occurred during ...,continue
3,D00F3A8D5F43B2,example with symptoms of heartburn (upper GI s...,deprescribe
4,D8017C77BA15FA,14d h pylori treatment if pt treated for 2wks ...,stop
5,DA15CCF42ACF8B,"Barrett's esophagus, continue PPI",continue
6,D45728A2EFD315,"example with esophagitis, continue PPI",continue
7,DFB07E6B8F0957,example with chronic NSAID use and pharmacist ...,stop
8,D1BD3665C06499,example where pt should be on due to history o...,continue
9,D068E26FFF9F43,example where pt came in on and no clear reaso...,stop


In [10]:
DL.get_diagnosis_data(encounter_key=encounter_key)

{'hospitalAcquiredDx': '[{"EncounterKey":"D8FCAB3DB25660","DxName":"*Unspecified"}]',
 'presentOnAdmitDx': '[{"EncounterKey":"D8FCAB3DB25660","DxName":"*Unspecified"}]'}

In [11]:
DL.get_encounter_data(encounter_key=encounter_key)

'[{"EncounterKey":"D8FCAB3DB25660","PatientKey":"D530CE1F6B8C88","Sex":"Male","Age_y":66,"BirthDate":"1956-11-13","PtAdmitDate":"2023-09-03","PtDischargeDate":"2023-09-03","DRG":"*Unspecified","FinancialClass":"Medi-Cal Managed Care","AdmissionOrigin":"ED Admission","AdmissionSource":"Self Referred (Home)","AdmissionType":"Emergency","PrimaryDx":"*Unspecified","PresentOnAdmissionDiagnosisComboKey":-1,"HospitalAcquiredDiagnosisComboKey":-1,"DischargeDisposition":"Home or Self Care","DischargePatientClass":"Inpatient"}]'

In [14]:
stuff = DL.get_notes_data(encounter_key=encounter_key)
stuff

,deid_note_key,EncounterKey,NoteDate,DepartmentSpecialty,ProviderSpecialty,ProviderType,NoteText,is_on_last_note_date,key,PtDischargeDate,discharge_text
0,D00061A8A25CE6,D8017C77BA15FA,2023-07-09,Inpatient Nursing,Nephrology,Physician,NEPHROLOGY FOLLOW UP CONSULT NOTE Events ...,0,D8017C77BA15FA,2023-09-04,None
1,D00243904093A0,D8017C77BA15FA,2023-07-19,Inpatient Nursing,Neurology,Physician Assistant,This is an independent service. The availa...,0,D8017C77BA15FA,2023-09-04,None
2,D004767E21ABBC,D8017C77BA15FA,2023-07-26,Inpatient Nursing,NaN,Nurse Practitioner,UCSF Medical Intensive Care Unit Progress No...,0,D8017C77BA15FA,2023-09-04,None
3,D005AD8F471521,D45728A2EFD315,2023-08-31,Inpatient Nursing,UCSF,Resident,Gastroenterology Initial Consultation Note ...,0,D45728A2EFD315,2023-09-17,None
4,D00B2936D4CF40,D0E44FD0BBD96F,2023-09-18,Inpatient Nursing,NaN,Registered Nurse,Problem: Pain Acute / Chronic- Adult Goal:...,0,D0E44FD0BBD96F,2023-09-22,None
...,...,...,...,...,...,...,...,...,...,...,...
1636,DFF89641837FF7,D45728A2EFD315,2023-08-26,Inpatient Nursing,Critical Care Medicine,Nurse Practitioner,ADVANCED LUNG DISEASE PROGRESS NOTE This is ...,0,D45728A2EFD315,2023-09-17,None
1637,DFFA2D7C1B2202,D4865B8BBB294E,2023-09-08,Inpatient Nursing,NaN,Nurse Practitioner,The following orders were created for panel or...,0,D4865B8BBB294E,2023-09-13,None
1638,DFFBAEF40F860B,D45728A2EFD315,2023-08-19,Inpatient Nursing,NaN,Registered Nurse,Problem: Discharge Planning - Adult Goal: K...,0,D45728A2EFD315,2023-09-17,None
1639,DFFC5634F74117,D45728A2EFD315,2023-09-16,Inpatient Nursing,NaN,Registered Nurse,Problem: Discharge Planning - Adult Goal: K...,0,D45728A2EFD315,2023-09-17,None


In [16]:
encounter_key

'D8FCAB3DB25660'

In [17]:
stuff[stuff["EncounterKey"] == 'D8017C77BA15FA']

,deid_note_key,EncounterKey,NoteDate,DepartmentSpecialty,ProviderSpecialty,ProviderType,NoteText,is_on_last_note_date,key,PtDischargeDate,discharge_text
0,D00061A8A25CE6,D8017C77BA15FA,2023-07-09,Inpatient Nursing,Nephrology,Physician,NEPHROLOGY FOLLOW UP CONSULT NOTE Events ...,0,D8017C77BA15FA,2023-09-04,None
1,D00243904093A0,D8017C77BA15FA,2023-07-19,Inpatient Nursing,Neurology,Physician Assistant,This is an independent service. The availa...,0,D8017C77BA15FA,2023-09-04,None
2,D004767E21ABBC,D8017C77BA15FA,2023-07-26,Inpatient Nursing,NaN,Nurse Practitioner,UCSF Medical Intensive Care Unit Progress No...,0,D8017C77BA15FA,2023-09-04,None
6,D00CD7440E4B25,D8017C77BA15FA,2023-07-09,Inpatient Nursing,UCSF,Resident,NEUROLOGY INITIAL CONSULT NOTE Consult req...,0,D8017C77BA15FA,2023-09-04,None
7,D00D4F104DFADA,D8017C77BA15FA,2023-07-11,Inpatient Nursing,NaN,Nurse Practitioner,"XR CHEST 1 VIEW 07/11/2023 9:19 AM"" HISTO...",0,D8017C77BA15FA,2023-09-04,None
...,...,...,...,...,...,...,...,...,...,...,...
1614,DFCD199D40486F,D8017C77BA15FA,2023-07-21,Inpatient Nursing,Infectious Diseases,Physician,TRANSPLANT INFECTIOUS DISEASES INITIAL CONSULT...,0,D8017C77BA15FA,2023-09-04,None
1620,DFDB1AA295F7C9,D8017C77BA15FA,2023-08-31,Inpatient Nursing,NaN,Registered Nurse,1840- Took over care from ***** ***** ***** ...,0,D8017C77BA15FA,2023-09-04,None
1624,DFE5FBC287079B,D8017C77BA15FA,2023-07-26,Inpatient Nursing,UCSF,Resident,Rheumatology Consult Service Follow-up Note ...,0,D8017C77BA15FA,2023-09-04,None
1628,DFE8CAE1C9179E,D8017C77BA15FA,2023-07-13,Inpatient Nursing,Cardiothoracic Surgery,Physician,THORACIC SURGERY ATTENDING ***** ***** NOTE ...,0,D8017C77BA15FA,2023-09-04,None
